<a href="https://colab.research.google.com/github/kahramanmurat/stock-markets-analytics-zoomcamp-2024/blob/main/02-dataframe-analysis/homework/Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt



---
### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happened on Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 25 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)

---

In [3]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipos_df = pd.read_html(response.text)

In [4]:
ipos_df = ipos_df[0]
ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     326 non-null    object
 1   Symbol          326 non-null    object
 2   Company Name    326 non-null    object
 3   Price Range     326 non-null    object
 4   Shares Offered  326 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB


In [5]:
# convert to datetime
ipos_df['Filing Date'] = pd.to_datetime(ipos_df['Filing Date'], format='%b %d, %Y')

In [6]:
ipos_df['Shares Offered'] = pd.to_numeric(ipos_df['Shares Offered'], errors='coerce')

In [7]:
import pandas as pd
import numpy as np

# Function to parse the price range and calculate the average
def calculate_average_price(price_range):
    # Handle cases where the cell might be empty or contain a dash
    if price_range.strip() == '-' or not price_range.strip():
        return np.nan
    # Remove the dollar sign and split by '-'
    parts = price_range.replace('$', '').split('-')
    # Convert parts to numeric, wrapped in a pandas Series
    numbers = pd.Series(pd.to_numeric(parts, errors='coerce'))
    # Filter out NaN values and compute the mean if any numbers are present
    valid_numbers = numbers.dropna()
    if valid_numbers.empty:
        return np.nan
    else:
        return valid_numbers.mean()

# Assuming ipos_df is your DataFrame and 'Price Range' is a column in it
# Apply the function to create the 'Average Price' column
ipos_df['Average Price'] = ipos_df['Price Range'].apply(calculate_average_price)

In [8]:
# Assuming 'ipos_df' is your DataFrame and it already contains 'Shares Offered' and 'Average Price' columns
ipos_df['Shares_offered_value'] = ipos_df['Shares Offered'] * ipos_df['Average Price']

In [9]:
# Filter rows where the filing date is on a Friday and in the year 2023
friday_filings = ipos_df[(ipos_df['Filing Date'].dt.dayofweek == 4) & (ipos_df['Filing Date'].dt.year == 2023)]

# Sum the 'Shares_offered_value' column
total_value = friday_filings['Shares_offered_value'].sum()

# Convert the total sum to millions of USD and round to the nearest integer
total_value_millions = round(total_value / 1e6)

# Display the result
print(f"Total value in millions of USD for all Friday filings during 2023: {total_value_millions} million USD")


Total value in millions of USD for all Friday filings during 2023: 286 million USD


In [10]:
friday_filings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 50 to 166
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           32 non-null     object        
 4   Shares Offered        25 non-null     float64       
 5   Average Price         25 non-null     float64       
 6   Shares_offered_value  25 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 2.0+ KB


---
### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this [link](https://stockanalysis.com/actions/changes/).
Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Additional:
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


---

In [11]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [12]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [13]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [14]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      65 non-null     object
 1   Symbol        65 non-null     object
 2   Company Name  65 non-null     object
 3   IPO Price     65 non-null     object
 4   Current       65 non-null     object
 5   Return        65 non-null     object
dtypes: object(6)
memory usage: 3.2+ KB


In [15]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [16]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,$5.19,29.75%


In [17]:
# Need to convert everything to a proper type (date, str, int, float, etc.)
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      219 non-null    object
 1   Symbol        219 non-null    object
 2   Company Name  219 non-null    object
 3   IPO Price     219 non-null    object
 4   Current       219 non-null    object
 5   Return        219 non-null    object
dtypes: object(6)
memory usage: 10.4+ KB


In [18]:
# convert to datetime
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')

In [19]:
stacked_ipos_df=stacked_ipos_df[(stacked_ipos_df['IPO Date'] < "2024-03-01") & (stacked_ipos_df['Symbol']!="RYZB")]

In [20]:
stacked_ipos_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
34,2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.69,167.25%
35,2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.14,-96.72%
36,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.16,1.60%
37,2024-02-16,CHRO,Chromocell Therapeutics Corp,$6.00,$1.75,-71.50%
38,2024-02-14,UMAC,"Unusual Machines, Inc.",$4.00,$1.08,-73.50%
...,...,...,...,...,...,...
214,2023-01-25,QSG,QuantaSing Group Ltd,$12.50,$3.01,-75.04%
215,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.49,-89.82%
216,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.16,147.73%
217,2023-01-13,ISRL,Israel Acquisitions Corp,$10.00,$10.93,9.30%


In [21]:
missing_prices_df = stacked_ipos_df[stacked_ipos_df['Current'].astype(str).str.find('-') >= 0]
missing_prices_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return


In [22]:
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')

In [23]:
stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')

In [24]:
# Convert 'Return' to numeric format (percentage)
stacked_ipos_df['Return'] = pd.to_numeric(stacked_ipos_df['Return'].str.replace('%', ''), errors='coerce') / 100


In [25]:
stacked_ipos_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
34,2024-02-27,SMXT,"SolarMax Technology, Inc.",4.00,10.69,1.6725
35,2024-02-22,VHAI,Vocodia Holdings Corp,4.25,0.14,-0.9672
36,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,10.00,10.16,0.0160
37,2024-02-16,CHRO,Chromocell Therapeutics Corp,6.00,1.75,-0.7150
38,2024-02-14,UMAC,"Unusual Machines, Inc.",4.00,1.08,-0.7350


In [26]:
import pandas as pd
import yfinance as yf



df = stacked_ipos_df

# Function to fetch historical data taking into account only weekdays
def fetch_historical_prices(symbol, ipo_date):
    try:
        start_date = pd.to_datetime(ipo_date)
        # Calculate end date considering only business days (weekdays)
        end_date = max(pd.bdate_range(start=start_date, periods=30))
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        if stock_data.empty:
            # Fetch maximum data available if no data found in the specific range
            stock_data = yf.download(symbol, period="max")
            # Ensure we only consider data up to the intended end_date
            stock_data = stock_data[stock_data.index <= end_date]
        return stock_data['Adj Close']
    except Exception as e:
        print(f"Failed to fetch data for {symbol}: {e}")
        return pd.Series([])  # Return an empty series in case of failure

# Generate future growth columns
for i in range(1, 31):
    df[f'growth_future_{i}d'] = None  # Initialize column

for index, row in df.iterrows():
    prices = fetch_historical_prices(row['Symbol'], row['IPO Date'])
    ipo_price = row['IPO Price']
    valid_days = pd.bdate_range(start=pd.to_datetime(row['IPO Date']), periods=31)[1:]  # Excluding IPO day
    for day in range(1, len(valid_days)+1):
        if valid_days[day-1] in prices.index:
            df.at[index, f'growth_future_{day}d'] = prices.loc[valid_days[day-1]] / ipo_price - 1

# Convert growth columns to numeric and handle NaNs
growth_columns = [f'growth_future_{i}d' for i in range(1, 31)]
df[growth_columns] = df[growth_columns].apply(pd.to_numeric, errors='coerce')  # Ensure all data is numeric
quantiles = df[growth_columns].quantile(0.75).dropna()

# Analyze the results, ensuring non-empty and valid data
if not quantiles.empty and quantiles.notna().any():
    optimal_x = quantiles.idxmax()
    highest_quantile_growth = quantiles.max()
    print(f"The optimal X is {optimal_x} with the highest 75% quantile growth of {highest_quantile_growth:.2%}")
else:
    print("No valid data available for analysis.")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['DYCQ']: Exception("%ticker%: Data doesn't exist for startDate = 1708491600, endDate = 1712030400")
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[********************

Failed to fetch data for MSS: 'MSS'
Failed to fetch data for ANL: 'ANL'
Failed to fetch data for SYRA: 'SYRA'


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Failed to fetch data for VSME: 'VSME'
Failed to fetch data for LRE: 'LRE'
Failed to fetch data for TURB: 'TURB'
Failed to fetch data for MDBH: 'MDBH'


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NNAG']: Exception("%ticker%: Data doesn't exist for startDate = 1692331200, endDate = 1695873600")
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[********************

The optimal X is growth_future_22d with the highest 75% quantile growth of 19.44%


In [27]:
df.describe()

,IPO Date,IPO Price,Current,Return,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
count,184,184.000000,184.000000,183.000000,131.000000,133.000000,131.000000,135.000000,138.000000,134.000000,...,138.000000,142.000000,145.000000,143.000000,146.000000,141.000000,142.000000,147.000000,0.0,0.0
mean,2023-08-01 01:57:23.478260992,10.449076,9.864674,-0.247029,2.015468,1.686811,0.852872,1.159804,1.282568,1.290861,...,0.758532,0.739148,0.703783,0.676618,0.618130,0.627191,0.204550,0.530495,NaN,NaN
min,2023-01-13 00:00:00,2.500000,0.000000,-0.999600,-0.950413,-0.952188,-0.953437,-0.953125,-0.954438,-0.953250,...,-0.953875,-0.955313,-0.952312,-0.940000,-0.963250,-0.965625,-0.965000,-0.964375,NaN,NaN
25%,2023-04-04 00:00:00,4.000000,1.197500,-0.742250,-0.176250,-0.220000,-0.229044,-0.278333,-0.346090,-0.384688,...,-0.463875,-0.463125,-0.477500,-0.462750,-0.459917,-0.471333,-0.472875,-0.488750,NaN,NaN
50%,2023-07-26 12:00:00,8.000000,5.545000,-0.410000,-0.011111,-0.043629,-0.029750,-0.025000,-0.022917,-0.075417,...,-0.058651,-0.048184,-0.059716,-0.068586,-0.091806,-0.090625,-0.099229,-0.102500,NaN,NaN
75%,2023-11-11 00:00:00,12.000000,10.780000,0.050150,0.153844,0.125625,0.149038,0.138500,0.134383,0.155573,...,0.171645,0.194408,0.134254,0.173986,0.126701,0.173714,0.121282,0.175221,NaN,NaN
max,2024-02-27 00:00:00,92.000000,118.270000,2.672500,129.500000,116.250000,91.833333,66.833333,88.666667,112.166667,...,62.500000,62.500000,64.333333,60.333333,57.333333,56.500000,27.500000,53.500000,NaN,NaN
std,NaN,9.894117,15.360169,0.647881,14.076477,12.228732,8.159369,8.212154,9.540009,10.500243,...,6.106750,5.971400,6.039863,5.681535,5.414901,5.353072,2.438712,4.934973,NaN,NaN


---
### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks
for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

---

In [28]:
import yfinance as yf
import pandas as pd

# Define the stock lists
LARGEST_STOCKS = [
    'MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO', 'V', 'JPM',
    'NVO', 'MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE', 'IDEXY', 'CDI.PA',
    'RELIANCE.NS', 'TCS.NS', 'HDB', 'BHARTIARTL.NS', 'IBN', 'SBIN.NS', 'LICI.NS', 'INFY', 'ITC.NS',
    'HINDUNILVR.NS', 'LT.NS'
]

NEW_US = ['TSLA', 'WMT', 'XOM', 'UNH', 'MA', 'PG', 'JNJ', 'MRK', 'HD', 'COST', 'ORCL']
NEW_EU = ['PRX.AS', 'CDI.PA', 'AIR.PA', 'SU.PA', 'ETN', 'SNY', 'BUD', 'DTE.DE', 'ALV.DE', 'MDT', 'AI.PA', 'EL.PA']
NEW_INDIA = ['BAJFINANCE.NS', 'MARUTI.NS', 'HCLTECH.NS', 'TATAMOTORS.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'ADANIENT.NS', 'NTPC.NS', 'KOTAKBANK.NS', 'TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

# Set date range for the data download
start_date = '2013-12-24'
end_date = '2023-12-31'

# Function to download stock data
def download_stock_data(tickers):
    data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
    return data

# Download data for both groups
largest_data = download_stock_data(LARGEST_STOCKS)
large_data = download_stock_data(LARGE_STOCKS)

# Calculate the 7-day growth for each stock and each day
def calculate_growth(data):
    return data.pct_change(7) + 1

largest_growth = calculate_growth(largest_data)
large_growth = calculate_growth(large_data)

# Average daily growth for each group
largest_daily_growth = largest_growth.mean(axis=1)
large_daily_growth = large_growth.mean(axis=1)

# Days when LARGE outperforms LARGEST
outperform_days = (large_daily_growth > largest_daily_growth).sum()

# Total number of trading days
total_days = len(largest_daily_growth)

# Calculate the percentage
percentage_outperform = (outperform_days / total_days) * 100

# Print the result
print(f"Percentage of days when LARGE stocks outperformed LARGEST stocks: {int(percentage_outperform)}%")



[*********************100%%**********************]  33 of 33 completed
[*********************100%%**********************]  33 of 33 completed


Percentage of days when LARGE stocks outperformed LARGEST stocks: 48%
